    %load_ext pidgin.shell
    
The `pidgin` shell augments the `IPython` shell in the notebook to permit Markdown forward [computational essays](http://blog.stephenwolfram.com/2017/11/what-is-a-computational-essay/).   It provides:

* Markdown input in code cells.
* Markdown block code statements and expressions are evaluated as a normal code cell.
* Inline code expressions are evaluated asynchronously.
* The Markdown code accepts `jinja2` templates too


In [1]:
    import IPython, traitlets, jinja2, types, contextlib, textwrap, ast, argparse, traceback
    try:
        from .tangle import markdown_to_python_user_expressions
    except:
        from tangle import markdown_to_python_user_expressions

In [2]:
    def compare_source(raw: str, source: str) -> bool: return list(
        map(str.rstrip, textwrap.dedent(raw).splitlines())
    ) == list(map(str.rstrip, textwrap.dedent(source).splitlines()))

In [8]:
    class PidginShell(IPython.core.interactiveshell.InteractiveShell):
        def run_cell(self, raw, store_history=False, silent=False, shell_futures=True, **user_expressions):
            """Run cell separates the sync and async parts."""
            self._last_traceback = None
            
            source, user_expressions = markdown_to_python_user_expressions(raw)
            
            result = IPython.core.interactiveshell.InteractiveShell.run_cell(
                self, source, store_history=store_history, 
                silent=silent, shell_futures=shell_futures)
            
            is_source =  compare_source(raw, source)
            try:
                is_display = bool(raw.partition('\n')[0].strip())
                if raw.strip() == ast.literal_eval(source).strip():
                    is_display = False
            except: ...
                                
            if is_display and not is_source: IPython.display.display(raw)
                
            self.user_expressions({str: str for str in user_expressions})
            return result
        
        def user_expressions(self, expressions):
            out = {}
            user_ns = self.user_ns
            global_ns = self.user_global_ns
            value = {}
            for key, expr in expressions.items():
                try:
                    value = self._format_user_obj(
                        eval(self.input_transformer_manager.transform_cell(expr), global_ns, user_ns))
                except:
                    self._user_obj_error()
                    value = None
                    break
            if value is None:
                traceback.print_last()
            return out        

In [9]:
    original_methods = {}
    def load_ipython_extension(ip):
        ip.run_cell = types.MethodType(PidginShell.run_cell, ip)
        ip.user_expressions = types.MethodType(PidginShell.user_expressions, ip)
        ip.enable_html_pager = True
    def unload_ipython_extension(ip):  
        ip.run_cell = types.MethodType(type(ip).run_cell, ip)

In [10]:
    parser = argparse.ArgumentParser(description="""Modify settings for a running pidgin kernel.""")

In [11]:
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())

{}
{}
